## Document Scanner

In this exercise, we will built a document scanner.

In [ ]:
# import the necessary packages
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils
import matplotlib.pyplot as plt

In [ ]:
# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it

image = cv2.imread("page.jpg")
#print(image.shape)

cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

ratio = image.shape[0] / 500.0
#print(ratio)

orig = image.copy()
image = imutils.resize(image, height = 500)
#print(image.shape)

In [ ]:
# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)

# show the original image and the edge detected image
print("STEP 1: Edge Detection")

In [ ]:
#cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
#cv2.namedWindow('Edged', cv2.WINDOW_NORMAL)
#cv2.imshow("Image", image)
#cv2.imshow("Edged", edged)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
#plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#plt.show()

#plt.imshow(gray, cmap = "gray")
#plt.show()

#plt.imshow(blurred, cmap = "gray")
#plt.show()

#plt.imshow(edged, cmap = "gray")
#plt.show()

In [ ]:
# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour

#cv2.findContours(image_source, counter retrieval mode, approximation)
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

#Additional Reference:
#https://docs.opencv.org/3.1.0/d4/d73/tutorial_py_contours_begin.html

#Try out following contour features later
#https://docs.opencv.org/3.1.0/dd/d49/tutorial_py_contour_features.html

In [ ]:
# loop over the contours
for c in cnts:
	# approximate the contour
    #cv2.arcLength(contour, is_open_contour)
	peri = cv2.arcLength(c, True)
    #cv2.approxPolyDP approximates a contour shape to another shape with less number of vertices depending upon the precision we specify.
    #Second argument in it is called epsilon, which is maximum distance from contour to approximated contour.
	approx = cv2.approxPolyDP(c, 0.02 * peri, True)

	# if our approximated contour has four points, then we
	# can assume that we have found our screen
	if len(approx) == 4:
		screenCnt = approx
		break

# show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")

In [ ]:
#cv2.namedWindow('Outline', cv2.WINDOW_NORMAL)
#cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
#cv2.imshow("Outline", image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
from transform import four_point_transform

# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)

# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)

'''
Thresholding is used to create a binary image from a grayscale image. 
It is the simplest way to segment objects from a background.

We binarize an image using the threshold_local function, 
which calculates thresholds in regions with a characteristic 
size block_size surrounding each pixel (i.e. local neighborhoods). 
Each threshold value is the weighted mean of the local neighborhood minus an offset value.
'''
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255

In [ ]:
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.namedWindow('Original', cv2.WINDOW_NORMAL)
cv2.namedWindow('Scanned', cv2.WINDOW_NORMAL)
cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)